# Testing the class with default files

## Testing default file inclusion

In [1]:
import numpy as np
import pandas as pd

from Bio.Align.Applications import MuscleCommandline
from Bio.GenBank import Record
import toytree

/Users/emilyleggat/miniconda3/lib/python3.12/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


Bio.Align.Applications gives a deprecation warning...

In [6]:
class Phydiv:
	"""
	This class takes community sequence (.fasta or .csv) and matrix occurence data (.csv) to
	produce measures of phylogenetic diversity and tree visualizations.
	"""

	default_seq = 'default_seq.csv'
	default_mat = 'default_mat.csv'


	def __init__(self, seq = default_seq, mat = default_mat):
		self.seq = seq #this can be .fasta or .csv
		self.mat = mat #this needs to be .csv
		# Include a parameter so either both are given, or both are left blank. Otherwise, an error

	def __repr__(self):
		return f"Phylogenetic diversity will be calculated using sequences in {self.seq} and communities in {self.mat}."


In [7]:
test = Phydiv()
test

Phylogenetic diversity will be calculated using sequences in default_seq.csv and communities in default_mat.csv.

In [17]:
class Phydiv:
    """
    This class takes community sequence (.fasta or .csv) and matrix occurence data (.csv) to
    produce measures of phylogenetic diversity and tree visualizations.
    """
    
    default_seq = pd.read_csv('default_seq.csv')
    default_mat = pd.read_csv('default_mat.csv')
    
    
    def __init__(self, seq = default_seq, mat = default_mat):
        self.seq = seq #this can be .fasta or .csv
        self.mat = mat #this needs to be .csv
        # Include a parameter so either both are given, or both are left blank. Otherwise, an error
    
    def __repr__(self):
        return f"Phylogenetic diversity will be calculated using sequences in {self.seq} and communities in {self.mat}."
    
    def csv_to_fasta(self):
        """
        A function to pull GenBank sequences from accession numbers if self.seq is a .csv
        """
        if self.seq[-4:] == ".csv":
            #Code to pull GenBank sequences from accession numbers and write a .fasta
            csv = pd.read_csv(self.seq)
        
        elif self.seq[-6:] == ".fasta":
            pass
    
        else: raise Exception("Sequence file must be .csv or .fasta")


In [23]:
test = Phydiv(seq = "cat")
test
test.csv_to_fasta()

Exception: Sequence file must be .csv or .fasta